# Capstone Project 
## Assignment 1 Part 1 
A Jupyter Note of the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [25]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

from bs4 import BeautifulSoup # xml parser library

print('Libraries imported.')

Libraries imported.


Get the Wikipedia page and create a BeautifulSoup object in order to parse and find the relevant information

In [26]:
wiki_page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(wiki_page, 'lxml') # BeautifulSoup object

# print(soup) reveals the first table is the postcode table 
table = soup.find('table') 


Build a list with all the row except cells with a borough that is Not assigned 

In [27]:
df_list = []
for row in table.find_all('tr')[1:]:  # skip the first row, which contains column headings
    row_list = []
    for item in row.find_all('td'):
        row_list.append(item.text.replace('\n',''))
    if row_list[1] != 'Not assigned': # Ignore cells with a borough that is Not assigned 
        df_list.append(row_list)


Group multiple neighborhoods with the same postcode

In [28]:
from itertools import groupby

df_list_unique = []
for key, group in groupby(df_list, lambda x: [x[0],x[1]]):
    neighs = ''
    for item in group: 
        neighs = neighs + item[2] + ', ' # grouping multiple items
    key.append(neighs[:-2])
    df_list_unique.append(key)


Now, build the pandas dataframe

In [29]:
column_names = ["PostalCode","Borough","Neighborhood"]

df = pd.DataFrame(df_list_unique, columns=column_names) # build the dataframe from the table
df.shape # show the shape

(103, 3)

## Assignment 1 Part 2 
Get the latitude and the longitude coordinates of each neighborhood/Postcode

##### Download Geospatial_Coordinates.csv

In [32]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

geo_df = pd.read_csv("Geospatial_Coordinates.csv", delimiter=",")

--2019-02-25 01:04:05--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-25 01:04:06--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-25 01:04:07--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

##### Adding latitude and longitude information for all postcodes

In [33]:
# add 2 empty columns
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# find and set latitude and longitude information for each postcode
for i in range(df.shape[0]):
    postcode = df.iat[i,0]
    lat_lng_coords = geo_df.loc[geo_df['Postal Code'] == postcode]
    df.iat[i,3] = lat_lng_coords['Latitude'].values[0]
    df.iat[i,4] = lat_lng_coords['Longitude'].values[0]
    
df.head(12) # checking

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
